In [84]:
from creds import (
    FOLDER_ID,
    OAUTH_TOKEN
)

import requests
import json
import pandas as pd
import time

In [108]:
FOLDER_ID = 'b1gh2vpl6n8dk3bdh7ou'



### connect to GPT

In [109]:
request = requests.post("https://iam.api.cloud.yandex.net/iam/v1/tokens", 
              json={"yandexPassportOauthToken" : OAUTH_TOKEN}, 
              )

IAM_TOKEN = request.json()['iamToken']
expiration_str = request.json()['expiresAt']
print("Token expires at", expiration_str[:10], expiration_str[11:16])

Token expires at 2024-03-27 02:05


In [110]:
instruction_text = f"""Ты работаешь продуктовым аналитиком в фаст-фуд компании. 
                    Выдели топ 3 конкретных рекомендаций по улучшению мобильного приложения для заказа еды на основе корпуса отзывов. 
                    Объясни подробно каждую данную тобой рекомендацию - почему она была тобой дана, на основе чего."""

In [111]:
request_text = """Не рекомендую заказывать через данное приложение, оформлял заказ, в итоге через полтора часа, в приложении срок доставки постоянно увеличивается; 
Не было ни одного раза когда при доставке положили все позиции, ни разу Даже при заказе в заведении не доклывают,отвратительно; 
Хочу оставить негативный отзыв, курьеры компании не доставили заказ, чеки все есть. Пришел даже разбирать в заведение где мой заказ, на что был культурно послан. А про поддержку я вообще молчу настолько проблемное приложение и люди в поддержке. Требовал вернуть средства уже неделю не читают сообщения.; 
Воруют деньги, если заказал не в тот ресторан. аргументируя тем, что срок годности истекает через 15 минут. Также сначала когда выбираю что-то из акции оно спрашивает, заказать ли по ближайшему адресу. Я нажимаю ага, а на деле оказывается адрес остаётся предыдущий и я заказываю не в тот ресторан. Недружелюбное поведение приложения зашкаливает; 
Приложение хорошее но, как это нет доставки на дом приходится заказывать и идти в вкусно-и точка и его забирать; Ужасный сервис доставки принимают заказ потом мурыжат пол часа и отменяют; Заказать доставку невозможно. По номеру горячей линии бот сбрасывает, в тех. поддержке игнор.; Взяли и не доложили 4 пирожка >:(; 
Доставка отвратительная. Хочу визу хочу нет; 
Нет доставки. Заказ только в сам ресторан.; 
Сделал заказ с доставкой на дом,деньги списали, заказа нет,на горячей линии телефон занят,деньги вернули спустя 10 дней.; 
По доставке не работает контакт центр, даже если, что то не довезли, как это обычно, обратится и решить проблему не где. Телефон не работает, на сообщение не отвечают .; Заказал доставку, доставки не было, деньги не вернули.; 
Заказала через приложение, через полчаса зашла посмотреть, когда привезут, а там написано: заказ аннулирован. В горячей линии сказали, что не хватило курьера. То есть нормально, что я жду заказ, а его просто так в любой момент могут аннулировать. В самом приложении не было ни бонусов, ни извинений, ни объяснения причины. Браво! Советую заказывать не через это приложение; 
Убрали доставку с другиз приложений, но при этом в своём не чего не до работали отменить заказ на стадии готовки нельзя, сменить адрес нельзя. Обратной связи нет. На звонки не ктотне отвечает."""

In [112]:
json_data_new = {
"model": "general",
"instruction_text": instruction_text,
"request_text": request_text,
"generation_options": {
                "max_tokens": 1500,  
                "temperature": 0.6
                }
}

In [113]:
json_file = '../data/prompt.json'

with open(json_file, "w") as fp:
    json.dump(json_data_new , fp)

with open(json_file) as json_data:
    prompt_date = json.dumps(json.load(json_data))

In [114]:
response = requests.post(
    "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct", 
    data=prompt_date,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {IAM_TOKEN}",
        "x-folder-id": f"{FOLDER_ID}",
    },
)

In [116]:
print(response.json()['result']['alternatives'][0]['text'])


1.	Сделать более наглядный интерфейс, который бы позволял пользователям, не имеющим опыта использования мобильных приложений, быстро разобраться в функционале и сделать заказ.
2.	Добавить возможность отслеживать статус заказа в режиме реального времени. Пользователи должны получать информацию о статусе заказа в виде уведомлений или чата с оператором.
3.	Улучшить систему лояльности для клиентов. Добавить возможность накапливать бонусы или скидки за каждый заказ. Это позволит привлечь новых клиентов и удержать старых.
4.	Внедрить функционал быстрой оплаты заказа. Клиенты должны иметь возможность оплатить заказ онлайн, чтобы избежать ожидания курьера с наличными.
5.	Устранить проблемы с доставкой. Обеспечить своевременное прибытие курьера и доставку заказа в указанное место.
6.	Расширить список ресторанов, которые можно заказать через мобильное приложение.
7.	Улучшить дизайн приложения. Сделать его более привлекательным и современным.
8.	Добавить функцию обратной связи. Дать возможность к

### Score scored data

In [85]:
sub_df = pd.read_csv('../data/scored.csv')

In [87]:
sub_df.head()

,Unnamed: 0,review,date,rating,preds,type_preds
0,0,Это приложение вообще умеет работать без ошибок?,2024-01-29 09:55:20,1,10,Неинформативно
1,1,"Убогая поделка, работает когда хочет, не хочет...",2024-01-29 09:54:21,1,10,Неинформативно
2,2,для такой большой сети - такое глюченное прило...,2024-01-29 09:47:38,1,10,Неинформативно
3,3,Невозможно залогиниться,2024-01-29 09:47:00,1,10,Неинформативно
4,4,Вообще не работает.,2024-01-29 09:43:58,1,10,Неинформативно


In [96]:
print(';\n'.join(sub_df[sub_df['type_preds'] == test_categories[0]]['review'].values[:15]))

1."При выборе партнёрского приза произошла ошибка" - при попытке забрать приз стикермании. На двух телефонах - рэдми 9 note и Samsung. Стикерманию выкатили в нерабочем виде, обновление приложения не помогает. 2. Невозможно забрать 6 раф по акции - тоже выдаёт ошибку, ни в терминале, ни девочки на прилавке выдачи - ничем не смогли помочь, использовать промокод на бесплатный напиток невозможно уже давно. 3. Вечно принудительно обновляется в самый неудобный момент, скажем, при сканировании кода.;
В приложении показываются акции которые не актуальны либо не работают. Информация а нем некорректная, хочешь заказать и приобрести одно, а по итогу берешь другое так как акция не действительна.;
Как айтишник, могу авторитетно судить о профнепригодности разработчиков этого приложения. Приложение в 12.00 работает, а в 16.00 когда ты уже голодный пытаешься сделать заказ у терминала, оно тебе сообщает, что необходимо обновление, и никакие операции в приложении невозможны. Вай-фай конечно не позволяет

In [127]:
sub_df.loc[sub_df.type_preds!='Неинформативно',['type_preds', 'rating']].groupby('type_preds').count().sort_values('rating')



,rating
type_preds,
Ассортимент/Качество еды,1
Обслуживание ПБО,1
Акции/Цены,2
Ошибка приложения,3
Регистрация в приложении,4
Локация ПБО,16
Доставка,23
Возврат денег/Отмена заказа,80
Функционал приложения,225


In [119]:
# тестирование API запроса к YaGPT

test_categories = list(set(sub_df.type_preds.unique().tolist()) - set(['Неинформативно']))

for test_category in test_categories:
    request_text = ';\n'.join(sub_df[sub_df['type_preds'] == test_category]['review'].values[:15]) # 45 отзывов максимум

    json_data_new = {
        "model": "general",
        "instruction_text": instruction_text,
        "request_text": request_text,
        "generation_options": {
        "max_tokens": 1500,  
        "temperature": 0.6
        }
    }


    json_file = '../data/prompt.json'

    with open(json_file, "w") as fp:
        json.dump(json_data_new , fp)

    with open(json_file) as json_data:
        prompt_date = json.dumps(json.load(json_data))

    # ------------------------------------

    response = requests.post(
        "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct", 
        data=prompt_date,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {IAM_TOKEN}",
            "x-folder-id": f"{FOLDER_ID}",
        },
    )

    print(f'Категория: {test_category}')
    # print(response.json())
    # print(response.json()['result'])
    print(response.json()['result']['alternatives'][0]['text'])
    print('-'*10)
    print()
    time.sleep(5)


# повторный заказ
# сохранение любимых ресторанов
# подключение соц. сетей
# регистрация через QR Code
# навигация - поиск товаров

Категория: Функционал приложения
1. Из отзывов становится понятно, что в мобильном приложении для заказа еды "Вкусно и точка" есть проблемы с акциями и промокодами. Также в отзывах указано, что информация в приложении может быть некорректной и пользователи сталкиваются с проблемами при попытке сделать заказ. Это может быть связано с тем, что акции и промокоды могут быть неактуальными или не работать корректно. Кроме того, пользователи жалуются на принудительное обновление приложения в самый неподходящий момент, что делает использование приложения неудобным.

Рекомендация: 

- Провести аудит акций и промокодов в мобильном приложении "Вкусно и точка". Необходимо убедиться, что они актуальны и работают корректно.
- Улучшить механизм отображения акций и промокодов в приложении. Он должен быть понятным и удобным для пользователей.
- Разработать систему уведомлений о новых акциях и промокодах, чтобы пользователи могли быстро и легко узнавать о них.
- Исключить принудительное обновление мобил